In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.providers.aer import AerProvider
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
#from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info.operators import Operator
import numpy as np
from math import pi
from matplotlib import pyplot as plt

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
#QUANTUM SIMULATOR FOR RUNNING QUANTUM RECURRENT NEURAL NETWORK ON IBM'S QUANTUM COMPUTERS
# REFERENCE: Gonçalves CP (2022) Quantum Neural Networks, Computational Field Theory and Dynamics. Int J Swarm Evol Comput. 11: 246.

transient=1000
T=50
r = 0.550129597
backend= #PLACE BACKEND HERE
num_shots=5000
    

I = np.matrix([[1,0],[0,1]])
V = np.matrix([[0,-1],[1,0]])
Ur=np.cos(r*pi/2)*I + np.sin(r*pi/2)*V
P0 = np.matrix([[1,0],[0,0]])
P1 = np.matrix([[0,0],[0,1]])

U01=np.matrix(np.kron(P0,I))+np.matrix(np.kron(P1,Ur))
U10=np.matrix(np.kron(I,P0))+np.matrix(np.kron(Ur,P1))

Map = np.dot(U01,U10)

state = np.matrix([1/2,1/2,1/2,1/2]).T
states = []

for t in range(0,transient):
    state = np.dot(Map,state)
   
for t in range(0,T):
    state = np.dot(Map,state)
    states.append(state)


P00=[]
P01=[]
P10=[]
P11=[]
N0 = []
N1 = []
counts_results=[]

for state in states:
    n0 = QuantumRegister(1,'n0')
    n1 = QuantumRegister(1,'n1')
    c = ClassicalRegister(2,'c')
    
    Circuit = QuantumCircuit()
    Circuit.add_register(n0)
    Circuit.add_register(n1)
    Circuit.add_register(c)
    
    
    Circuit.initialize(state.T.tolist()[0])
    
    Circuit.measure(n0,c[0])
    Circuit.measure(n1,c[1])
       
    # Get the backend
    if backend == 'qasm_simulator':
        device = Aer.get_backend(backend) # use Aer if the backend is the 'qasm_simulator' (default)
    else:
        device = provider.get_backend(backend) # use the provider otherwise
        
    
    # Execute the circuit on the device for the number of shots defined
    job = execute(Circuit, device, shots=num_shots)
    
     # Get the simulation results for the repeated experiments
    result = job.result()
    
    # Extract the counts, and return them for further processing
    counts = result.get_counts(Circuit)
    
    counts_results.append(counts)
    
    keys=counts.keys()
    
    C01 = 0
    C10 = 0
    C11 = 0
    
    
    if '00' in keys:
        P00.append(counts['00'])
    else:
        P00.append(0)
        
    if '01' in keys:
        C01 = counts['01']
        P01.append(counts['01'])
    else:
        C01 = 0
        P01.append(0)
    
    if '10' in keys:
        C10 = counts['10']
        P10.append(counts['10'])
    else:
        C10 = 0
        P10.append(0)
        
    if '11' in keys:
        C11 = counts['11']
        P11.append(counts['11'])
    else:
        C11 = 0
        P11.append(0)
    
    
    N0.append((C10+C11)/num_shots)
    N1.append((C01+C11)/num_shots)
    

print("\nP00")

for t in range(0,len(P00)):
    print(P00[t])


print("\nP01")

for t in range(0,len(P01)):
    print(P01[t])
    
print("\nP10")

for t in range(0,len(P10)):
    print(P10[t])

print("\nP11")

for t in range(0,len(P11)):
    print(P11[t])
    
print("\nN0")
for t in range(0,len(N0)):
    print(N0[t])

print("\nN1")
for t in range(0,len(N1)):
    print(N1[t]) 
    
plt.scatter(N0,N1,s=1)
plt.show()